# Fine Tuning LLMs using LLAMA-FACTORY
## Dataset Used
**Natural Language to Docker Command Dataset**
This dataset is designed to translate natural language instructions into Docker commands. It contains mappings of textual phrases to corresponding Docker commands, aiding in the development of models capable of understanding and translating user requests into executable Docker instructions.

Dataset Format
Each entry in the dataset consists of a JSON object with the following keys:
* input: The natural language phrase.
* instruction: A static field indicating the task to translate the phrase into a Docker command.
* output: The corresponding Docker command.

**1. Model Download and Initialization:**

- Downloading shards: Indicates the process of downloading the model's weight shards.
- Instantiating MistralForCausalLM model: The model is being instantiated with default data type torch.float16.
- Loading checkpoint shards: The model’s checkpoints are being loaded.
- All model checkpoint weights were used: Confirms that all weights from the checkpoint were successfully loaded into the model.

**2. Model Configuration:**

- Generate config GenerationConfig: A configuration object is created, specifying the beginning-of-sequence token (bos_token_id) and end-of-sequence token (eos_token_id).

**3. Gradient Checkpointing and Fine-tuning:**

- Gradient checkpointing enabled: This technique is used to reduce memory usage during training.
- Using torch SDPA for faster training and inference: Indicates the use of a specific method for efficient training and inference.
- Upcasting trainable params to float32: Trainable parameters are being upcasted to float32 for better precision during training.
- Fine-tuning method: LoRA: The model is fine-tuned using Low-Rank Adaptation (LoRA), a technique to reduce the number of trainable parameters.

**4. Training Details:**

- Num examples = 2,415: The number of training examples.
- Num Epochs = 1: The model is trained for one epoch.
- Batch size and optimization steps: Details about batch size and the number of optimization steps.
- Logging training progress: Loss, learning rate, gradient norm, and epoch progress are logged at regular intervals.

**5. Training Metrics:**

- Train loss and runtime metrics: At the end of training, the overall training loss and runtime metrics are reported.
- Saving model checkpoint: The trained model checkpoint is saved to a specified directory.

**6. Loading and Quantization:**

- Loading tokenizer and configuration files: These files are essential for the model to process inputs correctly.
- Quantizing model to 4 bit: Indicates that the model is being quantized to reduce its size and improve inference speed.

**7. Using the Model for Inference:**

- Loading weights and initializing the model: The model weights are loaded, and the model is initialized for inference.


In [1]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 259 (delta 46), reused 150 (delta 31), pack-reused 0
Receiving objects: 100% (259/259), 7.77 MiB | 15.07 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
cd '/content/LLaMA-Factory'

/content/LLaMA-Factory


In [3]:
!pip install -e .[torch,metrics]

Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 k

In [4]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00


In [5]:
!pip install --upgrade huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.1
    Uninstalling huggingface-hub-0.23.1:
      Successfully uninstalled huggingface-hub-0.23.1


In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [7]:
!CUDA_VISIBLE_DEVICES=0 GRADIO_SHARE=1 llamafactory-cli webui

2024-06-01 10:39:42.225883: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 10:39:42.225943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 10:39:42.227448: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-01 10:39:42.235330: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 10:39:43.747749: W tensorflow/compiler/tf2